In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate,FewShotChatMessagePromptTemplate
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

# ChatOpenAI 모델 초기화
chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

# 예시 대화 데이터 정의
examples = [
    {
        "question": "What is your location?",
        "answer": "Arrrrg! That be somewhere in the Caribbean, matey! But I won't tell ye more!"
    },
    {
        "question": "Do you have a pet?",
        "answer": "Aye! Got meself a faithful parrot named Salty! Squawks like a true pirate!"
    },
    {
        "question": "What's your favorite drink?",
        "answer": "Rum! Always rum, ye landlubber! Nothing better than a bottle o' rum at sea!"
    }
]

# 1. FewShotPromptTemplate 사용 예시
# 각 예시를 포맷팅할 템플릿 정의
example_prompt = PromptTemplate(
    input_variables=["question", "answer"],
    template="Human: {question}\nPirate: {answer}",
)

# FewShotPromptTemplate 설정
few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix="You are a pirate. Here are some examples of how you talk:",
    suffix="Human: {input}\nPirate:",
    input_variables=["input"],
    example_separator="\n\n"
)

# 2. FewShotChatMessagePromptTemplate 사용 예시
# 예시 메시지를 포맷팅할 템플릿 정의
example_chat_prompt = ChatPromptTemplate.from_messages([
    ("human", "{question}"),
    ("assistant", "{answer}")
])

few_shot_chat_prompt = ChatPromptTemplate.from_messages([
    # 시스템 메시지로 역할 설정
    SystemMessagePromptTemplate.from_template(
        "You are a pirate. You must speak like a stereotypical pirate in all responses."
    ),
    # Few-shot 예시들
    FewShotChatMessagePromptTemplate(
        examples=examples,
        example_prompt=example_chat_prompt,
        separator="\n\n"
    ),
    # 실제 사용자 입력을 위한 템플릿
    HumanMessagePromptTemplate.from_template("{input}")
])

# 두 가지 방식으로 체인 생성 및 실행
chain1 = few_shot_prompt | chat
chain2 = few_shot_chat_prompt | chat

# 실행 예시
print("=== FewShotPromptTemplate 결과 ===")
chain1.invoke({"input": "What's your favorite food?"})

print("\n=== FewShotChatMessagePromptTemplate 결과 ===")
chain2.invoke({"input": "What's your favorite food?"}) 

ValidationError: 1 validation error for FewShotChatMessagePromptTemplate
separator
  extra fields not permitted (type=value_error.extra)